# 💻 SQL Queries for Computer Equipment Pricing Dataset
This notebook contains useful SQL queries that can be used to explore and analyze the computer equipment pricing dataset.

In [1]:
import sqlite3
import pandas as pd


In [2]:

df = pd.read_csv('computer_equipment_prices.csv') 
# Coversion of Date Manufactured data type from object to datetime
df['Date Manufactured'] = pd.to_datetime(df['Date Manufactured'], errors='coerce')
# For product age in years
from datetime import datetime

today = pd.Timestamp.today()
df['Product Age (Years)'] = (today - df['Date Manufactured']).dt.days // 365
df.head()

# Create SQLite in-memory DB
conn = sqlite3.connect(':memory:')
df.to_sql('products', conn, index=False, if_exists='replace')


600

In [3]:
#Query 1 - Count of products by Brand
query1 = """
SELECT Brand, COUNT(*) AS Product_Count
FROM products
GROUP BY Brand
ORDER BY Product_Count DESC
"""
pd.read_sql_query(query1, conn)


,Brand,Product_Count
0,Kingston,39
1,Apple,35
2,TP-Link,34
3,ASUS,34
4,AMD,34
5,Netgear,32
6,MSI,32
7,Acer,32
8,Samsung,31
9,Dell,31


In [4]:
# Query 2 - Average price by Category
query2 = """
SELECT Category, ROUND(AVG([Price (USD)]), 2) AS Avg_Price
FROM products
GROUP BY Category
ORDER BY Avg_Price DESC
"""
pd.read_sql_query(query2, conn)


,Category,Avg_Price
0,Laptop,1656.68
1,Desktop,1440.05
2,Graphics Card,1244.18
3,Processor,602.20
4,Printer,562.97
5,Monitor,468.29
6,Motherboard,295.13
7,Router,279.16
8,Network Switch,278.63
9,SSD,204.23


In [5]:
#Query 3 - Top 5 suppliers with most stock
query3 = """
SELECT Supplier, SUM(Stock) AS Total_Stock
FROM products
GROUP BY Supplier
ORDER BY Total_Stock DESC
LIMIT 5
"""
pd.read_sql_query(query3, conn)


,Supplier,Total_Stock
0,TechWorld,35868
1,NextGen Hardware,30739
2,ComputeMart,30165
3,ITSupplies,28982
4,GadgetDepot,28013


In [6]:
# Query 4 - Oldest products (by year)
query4 = """
SELECT Brand, [Date Manufactured], [Product Age (Years)], [Price (USD)]
FROM products
ORDER BY [Product Age (Years)] DESC
LIMIT 5
"""
pd.read_sql_query(query4, conn)


,Brand,Date Manufactured,Product Age (Years),Price (USD)
0,Seagate,2013-04-15 00:00:00,12,67.40
1,HP,2013-02-17 00:00:00,12,2357.12
2,Apple,2013-04-26 00:00:00,12,69.44
3,Seagate,2013-01-21 00:00:00,12,411.83
4,Crucial,2013-05-19 00:00:00,12,389.18


In [7]:
# Query 5 - Count of products within different price ranges
query5 = """
SELECT 
  CASE 
    WHEN [Price (USD)] < 200 THEN 'Under $200'
    WHEN [Price (USD)] BETWEEN 200 AND 500 THEN '$200 - $500'
    WHEN [Price (USD)] BETWEEN 500 AND 1000 THEN '$500 - $1000'
    ELSE 'Over $1000'
  END AS Price_Range,
  COUNT(*) AS Count
FROM products
GROUP BY Price_Range
ORDER BY Count DESC
"""
pd.read_sql_query(query5, conn)


,Price_Range,Count
0,Under $200,278
1,$200 - $500,192
2,Over $1000,65
3,$500 - $1000,65


In [8]:
#Query 6 - Brands with average price above overall average
query6 = """
SELECT Brand, ROUND(AVG([Price (USD)]), 2) AS Avg_Price
FROM products
GROUP BY Brand
HAVING Avg_Price > (SELECT AVG([Price (USD)]) FROM products)
ORDER BY Avg_Price DESC
"""
pd.read_sql_query(query6, conn)


,Brand,Avg_Price
0,Corsair,662.40
1,Crucial,551.56
2,Dell,515.07
3,TP-Link,502.26
4,ASUS,480.42
5,HP,476.68
6,Lenovo,476.12
7,MSI,466.93
8,Intel,465.08


In [9]:
# Query 7: Category-wise stock and average price
query7 = """
SELECT 
  Category,
  SUM(Stock) AS Total_Stock,
  ROUND(AVG([Price (USD)]), 2) AS Avg_Price
FROM products
GROUP BY Category
ORDER BY Total_Stock DESC
"""
pd.read_sql_query(query7, conn)


,Category,Total_Stock,Avg_Price
0,Router,9190,279.16
1,USB Hub,9121,57.13
2,Keyboard,8861,115.24
3,External Hard Drive,8545,167.48
4,Monitor,8514,468.29
5,Docking Station,8245,177.45
6,Mouse,8139,83.25
7,Desktop,8115,1440.05
8,RAM,8035,202.80
9,Laptop,7722,1656.68


In [10]:
# Query 8: Correlation-like summary between product age and price
query8 = """
SELECT 
  [Product Age (Years)] AS Age,
  COUNT(*) AS Product_Count,
  ROUND(AVG([Price (USD)]), 2) AS Avg_Price
FROM products
GROUP BY Age
ORDER BY Age DESC
"""
pd.read_sql_query(query8, conn)


,Age,Product_Count,Avg_Price
0,12,25,439.75
1,11,51,508.80
2,10,65,328.96
3,9,65,428.34
4,8,61,406.75
5,7,53,495.71
6,6,47,352.64
7,5,55,385.35
8,4,44,393.65
9,3,59,410.29


In [11]:
# Query 9: Supplier-Category combo with average price
query9 = """
SELECT 
  Supplier,
  Category,
  ROUND(AVG([Price (USD)]), 2) AS Avg_Price,
  COUNT(*) AS Total_Products
FROM products
GROUP BY Supplier, Category
ORDER BY Avg_Price DESC
LIMIT 10
"""
pd.read_sql_query(query9, conn)


,Supplier,Category,Avg_Price,Total_Products
0,GadgetDepot,Laptop,1840.57,7
1,TechWorld,Laptop,1764.75,4
2,TechWorld,Graphics Card,1760.20,4
3,GadgetDepot,Graphics Card,1722.06,4
4,ITSupplies,Laptop,1646.35,5
5,NextGen Hardware,Desktop,1639.12,6
6,ITSupplies,Desktop,1627.73,4
7,ComputeMart,Laptop,1573.82,7
8,NextGen Hardware,Laptop,1535.82,9
9,GadgetDepot,Desktop,1533.51,5


In [12]:
# Find the top 3 most expensive products in each category using a window function
query10 = """
SELECT p.*
FROM products p
JOIN (
    SELECT Category, [Price (USD)],
           ROW_NUMBER() OVER (PARTITION BY Category ORDER BY [Price (USD)] DESC) AS rn
    FROM products
) ranked
ON p.Category = ranked.Category AND p.[Price (USD)] = ranked.[Price (USD)]
WHERE rn <= 3
ORDER BY Category, [Price (USD)] DESC
"""
pd.read_sql_query(query10, conn)


,Product ID,Category,Brand,Model,Price (USD),Stock,Supplier,Date Manufactured,Product Age (Years)
0,df1838bd-a775-4b5e-b460-3a3d4d577269,Desktop,ASUS,ASUS Desktop Model-7678,2483.08,99,GadgetDepot,2019-05-22 00:00:00,6
1,75c74d4f-ed6f-4036-91a5-c16568d7662f,Desktop,Lenovo,Lenovo Desktop Model-3844,2432.77,488,TechWorld,2022-04-28 00:00:00,3
2,c9456ac9-6ccc-45ed-96d4-91798af6e27e,Desktop,Corsair,Corsair Desktop Model-2322,2407.04,98,ITSupplies,2017-03-16 00:00:00,8
3,d5197d76-d01f-4895-9566-aaf24a6b897d,Docking Station,Kingston,Kingston Docking Station Model-6034,294.29,381,ITSupplies,2017-11-20 00:00:00,7
4,9a7bde8d-4af2-4f48-8453-b7f2131c1c1f,Docking Station,Kingston,Kingston Docking Station Model-6852,289.32,322,TechWorld,2014-11-01 00:00:00,10
5,76723393-a327-4899-9912-c4808de022e4,Docking Station,SteelSeries,SteelSeries Docking Station Model-6691,289.18,463,TechWorld,2022-12-09 00:00:00,2
6,5c0948bf-e306-4b17-963c-e7fb0a007b1e,External Hard Drive,Dell,Dell External Hard Drive Model-6062,298.93,208,GadgetDepot,2013-03-27 00:00:00,12
7,1fc5d2e3-77f9-4200-9dcd-446c61f65d14,External Hard Drive,Seagate,Seagate External Hard Drive Model-2386,285.14,178,NextGen Hardware,2015-11-29 00:00:00,9
8,9525e585-2e4e-45bd-8255-1b27f07f678f,External Hard Drive,Razer,Razer External Hard Drive Model-1725,276.70,60,ITSupplies,2022-06-27 00:00:00,2
9,00239cd4-66be-4e44-951e-d6f13c0e3e57,Graphics Card,TP-Link,TP-Link Graphics Card Model-1493,2407.92,17,TechWorld,2013-11-18 00:00:00,11


In [13]:
# Query 11: Brands with below-average pricing but above-average stock
query11 = """
SELECT Brand,
       ROUND(AVG([Price (USD)]), 2) AS Avg_Price,
       ROUND(AVG(Stock), 2) AS Avg_Stock
FROM products
GROUP BY Brand
HAVING Avg_Price < (SELECT AVG([Price (USD)]) FROM products)
   AND Avg_Stock > (SELECT AVG(Stock) FROM products)
ORDER BY Avg_Stock DESC
"""
pd.read_sql_query(query11, conn)


,Brand,Avg_Price,Avg_Stock
0,Seagate,358.07,316.26
1,AMD,290.91,288.59
2,SteelSeries,301.76,278.13
3,Kingston,362.46,276.87
4,Logitech,273.77,268.83
5,Samsung,376.74,258.00
6,Western Digital,379.33,257.28


In [14]:
# Query 12: Most diverse suppliers (offering most distinct categories)
query12 = """
SELECT Supplier,
       COUNT(DISTINCT Category) AS Distinct_Categories,
       COUNT(*) AS Total_Products
FROM products
GROUP BY Supplier
ORDER BY Distinct_Categories DESC
LIMIT 5
"""
pd.read_sql_query(query12, conn)


,Supplier,Distinct_Categories,Total_Products
0,TechWorld,20,134
1,NextGen Hardware,20,116
2,ITSupplies,20,120
3,GadgetDepot,20,112
4,ComputeMart,19,118
